In [ ]:
import numpy as np
import qokit
import scipy.optimize
from qokit import parameter_utils
import qokit.fur 

# Optimize QAOA parameters for LABS problem

We'll show an example of optimizing overlap with the ground state. Expectation value optimization is analogous.


First, let's get known parameters for largest depth $p$ that we have. QOKit has a built-in set of good LABS parameters.

In [ ]:
N = 10
parameters = parameter_utils.get_best_known_parameters_for_LABS_wrt_overlap(N)
known_p = parameters.p.max()
print(f"Maximum p available for N={N} is {known_p}")
parameters.head()

Now that we have the parameters for `p_known`, let's find parameters for a larger depth! We will extrapolate the parameters that we already know to have a good starting point for optimization

In [ ]:
p = known_p + 1
gamma, beta = parameter_utils.get_best_known_parameters_for_LABS_wrt_overlap_for_p(N, known_p)

The next cell will find an initial guess and create a function which maps parameters to the overlap. We can use `get_qaoa_labs_objective` with different parameterizations, but in this case, we'll use Fourier-transformed basis. 

The simulation will run on GPU if it is available, and if the problem is small, you will get a `NumbaPerformanceWarning`, which just indicates that you're not saturating capabilities of your GPU.

In [ ]:
u, v = parameter_utils.to_fourier_basis(gamma, beta)
init_u, init_v = parameter_utils.extrapolate_parameters_in_fourier_basis(u, v, p, 1)
init_freq = np.hstack([init_u, init_v])
f = qokit.get_qaoa_labs_objective(N, p, parameterization='freq', objective='overlap')

In [ ]:
print(f"Success probability at p={p} before optimization is {1-f(init_freq)}")

We will try optimizing further!

Specifically, we will extrapolate the parameters in the fourier basis using the interpolation scheme of https://arxiv.org/abs/1812.01041 and then optimize in the fourier basis

In [ ]:
res = scipy.optimize.minimize(f, init_freq, method='COBYLA', options={'rhobeg': 0.01/N})

In [ ]:
beta, gamma = parameter_utils.from_fourier_basis(res.x[:p], res.x[p:])
f_bg = qokit.get_qaoa_labs_objective(N, p, parameterization='gamma beta', objective='overlap')

print(f'Success probability at p={p} after optimization is {1-f_bg(gamma, beta)}')

Now we can use these parameters to solve LABS on a quantum computer!

---